### Conflict on Wikipedia

In this notebook, we will study reverts on Wikipedia. A revert can be understood as the event when an editor undoes the changes made by another editor (this is different from adding to what the other authors did). We will first identify who reverted whom and when, then identify situations in which if $A$ reverted $B$, $B$ reverted $A$ within 24 hours, and finally, study the difference in seniority between $A$ and $B$.

The goal here is to use basic Python as much as possible to showcase the uses of the different data types in Python. We could solve everything here using Objected-Oriented Programming or Procedural Programming (this is the one we will be using here, so that it's easier to follow the steps). 

##### A quick note on Data
We will use data from the file `../data/rowiki_2006.txt` in the `data` repository on GitHub. 
To data were originally obtained from the Wikipedia XML Dumps (https://dumps.wikimedia.org/mirrors.html) and include every article edit made on Romanian Wikipedia since it began until the end of 2006. Each line in the file is an edit and includes the title of the edited article, the time when the edit was submitted, whether the edit was a revert, the version of the article, and the user who submitted the edit. To detect the article versions, a hash was calculated for the complete article text following each revision and the hashes were compared between edits. 

The table below describes the variables in the data:

| Variable   | Explanation   
|:-----------|:-------
| title      | title of the edited article               
| time       | time in the format YYYY-MM-DD HH:MM:SS when the edit was completed  
| revert     | 1 if the edit was detected to revert to a previous article version, 0 otherwise 
| version    | an integer indicating a unique state of the article, generally increasing over time; -1 indicates the article was empty (usually due to vandalism); if the same number appears more than once, then the article was exactly in the same state at these different time points  
| user       | the editor's username or if not logged in, the editor's IP address  

### 1. Who reverted whom?

Here, we will create a network (e.g., an edge list), where an edge goes from the editor who restored an earlier version of the article (the "reverter") to the editor who made the revision immediately after that version (the "reverted"). For every edge, we will keep track of who the reverter was, who got reverted, when the revert occurred, and what the "seniority" of the the reverter and the reverted were at this point in time. Seniority here is the number of edits that the editor in question made up until that point. 

We will ignore the article titles for the analyses so we don't have to save this. To make things a little easier, we will also avoid self-reverts (when an author reverts his own version of the article).

We will estimate seniority $s_i$ of editor $i$ as the base-ten logarithm of the number of edits $i$ has completed by the time of the revert under question. Transforming the number of edits with the logarithm makes sense because they follow a power-law distribution (the majority of individuals have very few edits, while a handful of individuals are responsible for most of the work). This operationalization allows to express the difference in seniority between two editors as the base-ten logarithm of the ratio of number of edits since $s_i - s_j = \log_{10} e_i - \log_{10} e_j = \log_{10} \frac{e_i}{e_j}$, where $e_i$ is the number of edits of editor $i$ and $e_j$ is the number of edits of editor $j$. In essence, we assume that an editor who has 10 edits compares to one with 100 edits the same way that an editor with 1,000 edits compares to one with 10,000.

**Note that I write all of my code in the `reverts_network` module**, so you can have a look at this in the repository to see what the functions are doing exactly! Remember that it's good practice to keep your functions in a module, and even better so to keep related functions in related modules. Imagine that you were doing a really long project and using different functions that are unrelated to each other, you want to be able to access these quickly by importing them from the relevant module, without overloading your memory with a bunch of functions you don't need! This is why we want to practice good **repository hygiene**. 

If you're interested, you can read more about it here: https://docs.platform.sh/learn/bestpractices/clean-repository.html

In [ ]:
# Calling the functions from the network module 
# see here, we're calling the entire module that we need because we have just one, 
# but hopefully you can see how we might want to call just one function from a module, or different modules
import reverts_network as rn
file_path = "../data/rowiki_2006.txt"
skipped_lines = 1

# read the data file, then create the user info and then make the network
all_edits = rn.read_data(file_path, skipped_lines)
edit_counts, user_info_dict = rn.create_user_info(all_edits)
unique_users, network = rn.create_revert_network(all_edits, user_info_dict)

# Printing the results of the network 
print(f"The network has {len(unique_users)} nodes")
print(f"The network has {len(network)} edges")
print(network[:5])

### 2. If $A$ reverted $B$, did $B$ revert $A$?

Now that we have a handy network, we want to unravel some interesting social facts from it. The aim is to identify the two-event sequence in which after $A$ reverts $B$, $B$ reverts $A$ back ($AB–BA$) within 24 hours. To identify these sequences, use the revert data from the previous task. We will ignore the article titles — we are agnostic as to whether the response happens in the same or in different articles. This means that we no longer care if they revert each other on one article or many. If you know anything about Wikipedia authors, this makes sense because there are many bots, whose only aim is to revert others!

Further, it does not matter whether the response occurs immediately after the original revert or alternatively, the reverter and the reverted are involved in other reverts in-between the original revert and the response. If more than one response occurs within 24 hours, consider only the first one as part of the event sequence.

We also need to make sure that we keep track in the overall network of whom these authors are, so that we can later identify them easily.

In [ ]:
# Printing the results of the function count_ab_ba_sequences
# Function returns the number of AB-BA event sequences in the data
# The function also makes a new network to add AB-BA = True on nodes that are part of AB-BA event sequences
# and AB-BA = False on nodes that are not part of AB-BA event sequences
# BA parts of AB-BA sequences can be re-used multiple times (this increases the number of possible AB-BA sequences)
ab_ba_sequences_count, network_sorted = rn.count_and_label_AB_BA(network)
print(f"The number of AB-BA event sequences in the data is: {ab_ba_sequences_count}")

**Note:** make sure to have a look at the functions in the module to see what they do, I write a lot of comments there!


### 3. Are $A$ and $B$ more similar in seniority than expected?

Social comparison theory states that people strive to gain accurate self-evaluations and as a result, they tend to compare themselves to those who are similar. But since focus on relative performance heightens feelings of competitiveness, rivalry is stronger among similar individuals. This leads us to expect that the editors involved in the $AB–BA$ event sequences tend to be closer in seniority than expected.

To find evidence that retaliation is more likely among similar individuals, we will compare the *absolute difference* in seniority between editors involved in $AB–BA$ event sequences with the absolute difference in seniority between editors involved in any other revert (all the reverts in the data that are not part of the AB–BA sequences).

Let's **plot a histogram** of $|s_A - s_B|$ for reverts that are part of $AB–BA$ motifs on top of a histogram of $|s_i - s_j|$ for all other reverts. 

Then, let's compare the means, to see if there are any differences. 


**First** we'll need the absolute difference between the groups: 

In [ ]:
# Get absolute seniority difference for reverts in AB-BA sequences and the rest
# Initialize lists to store the absolute differences in seniority
ab_ba_diffs, other_diffs = rn.get_seniority_diffs(network_sorted)

**Second** we can plot the histogram. Note that I adjust the axes a little so that we can see the distributions better (one was otherwise very comparatively flat and hard to see).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# figure and axes for Matplotlib
fig, ax1 = plt.subplots()

plt.title('Histogram of Absolute Difference in Seniority between Reverter and Reverted for AB-BA Sequences and other Reverts')

# Plot the histogram for 'other_diffs' on the first Y-axis
ax1.hist(other_diffs, bins=30, alpha=0.5, color='blue', label='Other Reverts')
ax1.set_xlabel('Absolute Difference in Seniority of Reverter and Reverted')
ax1.set_ylabel('Frequency', color='blue')
ax1.tick_params('y', colors='blue')

# Create a second Y-axis and plot the histogram for 'ab_ba_diffs' on it
ax2 = ax1.twinx()
ax2.hist(ab_ba_diffs, bins=30, alpha=0.5, color='hotpink', label='AB-BA Reverts')
ax2.set_ylabel('Frequency', color='hotpink')
ax2.tick_params('y', colors='hotpink')

# Create a legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

fig.tight_layout()
plt.show()

# Print mean differences
mean_ab_ba_diff = round(np.mean(ab_ba_diffs), 2)
mean_other_diff = round(np.mean(other_diffs), 2)

print(f"Mean |s_A - s_B| for AB-BA motifs: {mean_ab_ba_diff}")
print(f"Mean |s_i - s_j| for other reverts: {mean_other_diff}")